<a href="https://colab.research.google.com/github/annabelcoates/redev_machine_learning_samples/blob/master/fp_train_odds_form_position_hda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries


In [0]:
import sys
import os
import urllib.request
import datetime;
import pandas as pd
import numpy as np
import math

from keras import optimizers
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda
from keras.utils import np_utils, to_categorical
from keras.utils import plot_model


# Load and Display Preprocessed Data 

For data without categorical names use preprocessed_data.csv as the filename or preprocessed_data_Cat.csv for the (220 columns of) categoric names

In [0]:

url='https://raw.githubusercontent.com/RedevLtd/redev_machine_learning_samples/master/FootballPredicator/preprocessed_data/efl_data_2000_2019position.csv'
df=pd.read_csv(url)
df_array=np.asarray(df)


In [36]:
count=0
for column in df.columns:
  print(str(count) +') '+ str(column))
  count+=1
  

0) FDIndex
1) Div
2) MatchNo
3) SeasonYear
4) SeasonMatchNo
5) MatchDate
6) HomeTeam
7) AwayTeam
8) OddsHomeWin
9) OddsDraw
10) OddsAwayWin
11) OddsWinDiff
12) CPHomeTeam
13) CPAwayTeam
14) L_FTHomeGoals
15) L_FTAwayGoals
16) L_FTResult
17) L_FTHomeWin
18) L_FTDraw
19) L_FTAwayWin
20) L_HTHomeGoals
21) L_HTAwayGoals
22) L_HTResult
23) L_HTHomeWin
24) L_HTDraw
25) L_HTAwayWin
26) L_HomePoints
27) L_AwayPoints
28) L_BTTS
29) L_FTHomeWin_BTTS
30) L_FTDraw_BTTS
31) L_FTAwayWin_BTTS
32) L_FTHomeWin_LHT
33) L_FTAwayWin_LHT
34) L_FTHomeWin_O25
35) L_FTDraw_O25
36) L_FTAwayWin_O25
37) HomeTeamPosition
38) AwayTeamPosition
39) HomeTeamF1
40) HomeTeamF2
41) HomeTeamF3
42) HomeTeamF4
43) AwayTeamF1
44) AwayTeamF2
45) AwayTeamF3
46) AwayTeamF4


In [37]:
df.tail(3)

,FDIndex,Div,MatchNo,SeasonYear,SeasonMatchNo,MatchDate,HomeTeam,AwayTeam,OddsHomeWin,OddsDraw,OddsAwayWin,OddsWinDiff,CPHomeTeam,CPAwayTeam,L_FTHomeGoals,L_FTAwayGoals,L_FTResult,L_FTHomeWin,L_FTDraw,L_FTAwayWin,L_HTHomeGoals,L_HTAwayGoals,L_HTResult,L_HTHomeWin,L_HTDraw,L_HTAwayWin,L_HomePoints,L_AwayPoints,L_BTTS,L_FTHomeWin_BTTS,L_FTDraw_BTTS,L_FTAwayWin_BTTS,L_FTHomeWin_LHT,L_FTAwayWin_LHT,L_FTHomeWin_O25,L_FTDraw_O25,L_FTAwayWin_O25,HomeTeamPosition,AwayTeamPosition,HomeTeamF1,HomeTeamF2,HomeTeamF3,HomeTeamF4,AwayTeamF1,AwayTeamF2,AwayTeamF3,AwayTeamF4
38129,E320228130,E3,20228130,2019,38130,2019-08-17,Northampton,Macclesfield,1.85,3.5,4.2,2.35,1,3,1.0,2.0,A,0,0,1,0.0,1.0,A,0,0,1,0,3,1,0,0,1,0,0,0,0,1,16.0,12.0,1,0,3,1,3,0,1,3
38130,E320228131,E3,20228131,2019,38131,2019-08-17,Salford,Port Vale,2.10,3.3,3.6,1.50,3,2,1.0,1.0,D,0,1,0,0.0,0.0,D,0,1,0,1,1,1,0,1,0,0,0,0,0,0,20.0,19.0,0,3,1,1,1,1,1,0
38131,E320228132,E3,20228132,2019,38132,2019-08-17,Scunthorpe,Crawley Town,1.95,3.4,3.9,1.95,0,3,2.0,2.0,D,0,1,0,1.0,1.0,D,0,1,0,1,1,1,0,1,0,0,0,0,1,0,21.0,6.0,0,0,0,0,3,0,3,0


# Rebalance and Shuffle the Data 

Rebalancing the data means making the number of instances of each result equal,which can make the model more accurate. 

In [0]:
# Seperate the data into draw, home win and away win results and crop each 
# dataframe down to the number of draws so that the data is balanced
#draws_df=df[df.L_FTResult=="D"]
#home_df=df[df.L_FTResult=="H"]
#home_df=home_df.iloc[range(draws_df.shape[0])]
#away_df=df[df.L_FTResult=="A"]
#away_df=away_df.iloc[range(away_df.shape[0])]
#concat_objs=[draws_df,home_df,away_df]
# Put the dataframes back together
#df=pd.concat(concat_objs,axis=0)

Shuffling the data avoids overfitting. The `df.sample(frac)` function returns a fraction of the dataframe as a random sample

In [0]:
# Shuffle the data
df=df.sample(frac=1)

# Get Input Data and Results

In [0]:
# For robustness call the odds and results by their dataframe column names instead of their indexes (in case the dataframe changes)
input_objs=[df.OddsHomeWin,df.OddsDraw,df.OddsAwayWin,df.HomeTeamPosition,df.AwayTeamPosition,
            df.HomeTeamF1,df.HomeTeamF2,df.HomeTeamF3,df.HomeTeamF4,df.AwayTeamF1,df.AwayTeamF2,df.AwayTeamF3,df.AwayTeamF4] 
input_array=np.asarray(pd.concat(input_objs,axis=1))
# Get the results
results_objs=[df.L_FTHomeWin,df.L_FTDraw,df.L_FTAwayWin]
results=np.asarray(pd.concat(results_objs,axis=1))

Print one data point of input and results to check it is in the expected form.

In [41]:
print(input_array[0])
print(results[0])

[ 1.91  3.3   4.2  17.   13.    3.    3.    3.    3.    3.    0.    3.
  3.  ]
[1 0 0]


In [0]:
# Get the number of inputs and the number of datapoints
number_dp=input_array.shape[0]
n_inputs= input_array.shape[1] 

In [0]:
input_array=input_array.reshape(number_dp,1,1,n_inputs)
results=results.reshape(number_dp,1,1,3)

# Seperate into training and testing data

Usually approx 80% of the data is used as training data and 20% of the data is used as test data

In [0]:
p=0.6
last_train= math.ceil(p*number_dp)
# Training data
input_train=input_array[0:last_train,:]
results_train=results[0:last_train,:]
# Test data
input_test=input_array[(last_train+1):number_dp,:]
results_test=results[(last_train+1):number_dp,:]

In [45]:
print(input_train.shape)
print(input_test.shape)

(22880, 1, 1, 13)
(15251, 1, 1, 13)


Print input shape

# Define Model Architecture

In [0]:
model=Sequential()
model.add(Dense(9,input_shape=(1,1,n_inputs), activation='relu')) # Adds a densely connected NN layer
model.add(Dense(3,activation='relu')) 
model.add(Dense(3,activation='softmax'))

# Compile Model

Categorial crossentropy loss is a loss algorithm for when there are multiple categories to sort each data point into, in this case  home win, draw or away win. 

In [0]:
#opt = SGD(lr=0.000001,momentum=0.004)
model.compile(loss='categorical_crossentropy',optimizer='adagrad',metrics=['accuracy'],sample_weight_mode="temporal") 
# Want to know the accuracy of the model so specify it as a metric
plot_model(model,to_file='model.png')

# Fit Model from Training Data


* The number of epochs are the number of phases of training
* The shuffle parameter determines whether the order of the batches is shuffled between epochs
* Batch size is the number of data points between updates to the weights
* **Greater number of epochs** can increase accuracy

Note: Rerunning this section will add to the current model, to fit a model from new recompile it by running the previous cell first



In [48]:
model.fit(x=input_train, y=results_train, batch_size=1000,
          epochs=100, verbose=1, shuffle=1)

Epoch 1/100
22880/22880 [==============================] - 0s 20us/step - loss: 1.5787 - acc: 0.2986
Epoch 2/100
22880/22880 [==============================] - 0s 6us/step - loss: 1.1314 - acc: 0.3756
Epoch 3/100
22880/22880 [==============================] - 0s 7us/step - loss: 1.0895 - acc: 0.4092
Epoch 4/100
22880/22880 [==============================] - 0s 6us/step - loss: 1.0756 - acc: 0.4246
Epoch 5/100
22880/22880 [==============================] - 0s 6us/step - loss: 1.0677 - acc: 0.4338
Epoch 6/100
22880/22880 [==============================] - 0s 6us/step - loss: 1.0621 - acc: 0.4409
Epoch 7/100
22880/22880 [==============================] - 0s 7us/step - loss: 1.0583 - acc: 0.4441
Epoch 8/100
22880/22880 [==============================] - 0s 6us/step - loss: 1.0554 - acc: 0.4477
Epoch 9/100
22880/22880 [==============================] - 0s 6us/step - loss: 1.0532 - acc: 0.4502
Epoch 10/100
22880/22880 [==============================] - 0s 7us/step - loss: 1.0514 - acc: 0.452

# Evaluate Model

In [49]:
[loss_value,accuracy]=model.evaluate(input_test,results_test,verbose=1)
print("The loss value for the model is "+ str(loss_value))
print("The accuracy value for the model is "+"{:.3f}".format(accuracy))

15251/15251 [==============================] - 2s 133us/step
The loss value for the model is 1.036437186942508
The accuracy value for the model is 0.474
